In [1]:
import os
import tensorflow as tf
import numpy as np

In [2]:
with open('notes', 'r') as file:
    notes = [int(x) for x in file.read().split()]

In [3]:
BATCH_SIZE = 10
S_LEN = 5 # sequence length
D = 12 # number of notes

In [4]:
data = tf.placeholder(tf.float32, [None, None, D])
target = tf.placeholder(tf.float32, [None, None, D])

In [5]:
num_hidden = 24
cell = tf.contrib.rnn.LSTMCell(num_hidden, state_is_tuple=True)

In [6]:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [7]:
W = tf.Variable(tf.random_normal([num_hidden, D], stddev=1.0/(num_hidden ** 0.5)))
b = tf.Variable(tf.zeros([D]))

In [12]:
y = tf.tensordot(val, W, axes=[[2], [0]])

In [13]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=target))

In [14]:
#optimizer = tf.train.GradientDescentOptimizer(1).minimize(loss)
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [30]:
def one_hot(x):
    v = np.zeros(D)
    if x > -1:
        v[x] = 1
    return v

In [31]:
lookup = {'': -1, 'C':0, 'Db':1, 'D':2, 'Eb':3, 'E':4, 'F':5, 'Gb':6, 'G':7, 'Ab':8, 'A':9, 'Bb':10, 'B':11}
def encode(seq):
    return np.array([[one_hot(lookup[x]) for x in seq]])
def decode(i):
    for n, j in lookup.items():
        if j == i:
            return n

In [18]:
def generate_sample(notes, length):
    while True:
        start = np.random.randint(0, len(notes)-length-1)
        yield np.array([one_hot(x) for x in notes[start:start+length]]), np.array([one_hot(x) for x in notes[start+1:start+length+1]])

In [19]:
def generate_batch(iterator, batch_size):
    while True:
        z = np.array([next(single_gen) for _ in range(batch_size)])
        yield z[:,0,:,:], z[:,1,:,:]

In [20]:
single_gen = generate_sample(notes, S_LEN)

In [21]:
batch_gen = generate_batch(single_gen, BATCH_SIZE)

In [22]:
saver = tf.train.Saver()

In [ ]:
# train
SKIP_STEP = 500
SKIP_STEP_N = 0
NUM_ITERATIONS = 1000000
with tf.Session() as sess:
    
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print('model restored')
    else:
        sess.run(tf.global_variables_initializer())
        print('model initialized')
    
    avg_loss = 0
    for i in range(NUM_ITERATIONS):

        a, b = next(batch_gen)
        _ , loss_batch, _ = sess.run([y, loss, optimizer], feed_dict={data:a, target:b})
        avg_loss += loss_batch
        
        if (i+1) % SKIP_STEP == 0:
            print(SKIP_STEP_N, avg_loss/SKIP_STEP)
            avg_loss = 0
            SKIP_STEP_N += 1
            saver.save(sess, "checkpoints/model.ckpt")

model restored
0 2.08082461977
1 2.06976170468
2 2.05909202862
3 2.05808918262
4 2.07373003888
5 2.06020962334
6 2.06917147732
7 2.06910712552
8 2.07326132464
9 2.07948147321
10 2.06137104082


In [78]:
# look at probability of next notes
with tf.Session() as sess:
    
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print('model loaded')
        
        test = ['D', 'E', 'Gb', 'G', 'A']
        [z] = sess.run([tf.nn.softmax(y)], feed_dict={data:encode(test)})
        z = z[0, -1, :]
        for i in reversed(z.argsort()):
            print(decode(i), z[i])

model loaded
B 0.411546
G 0.204688
A 0.0978602
Bb 0.0769005
Gb 0.063734
D 0.0586538
E 0.0247141
C 0.0243797
Ab 0.01209
Db 0.0118759
F 0.0108458
Eb 0.00271214


In [48]:
with tf.Session() as sess:
    
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print('model loaded')
        
        test = ['Eb']
        for _ in range(100):
            [z] = sess.run([tf.nn.softmax(y)], feed_dict={data:encode(test)})
            z = z[0, -1, :]
            next_note = np.random.choice(D, p=z)
            test.append(decode(next_note))
        print(' '.join(test))
    

model loaded
Eb D D G A G D G C C Eb Bb Bb Eb F C Eb Eb G C D C F Eb G Eb G B B D Gb A G Gb C E E C A D Eb Eb Eb Eb G C G C D G G G A G B Eb D G C G B D G Gb A G A C G B D G Gb E Gb D Gb A D Db B A Db D Db B Db D Db D Db D Db D Eb Gb E Eb D E Gb


In [50]:
import mido

In [60]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)
delta = 300
for n in [lookup[x] + 60 for x in test]:
    track.append(mido.Message('note_on', note=n, velocity=64, time=0))
    track.append(mido.Message('note_on', note=n, velocity=0, time=delta))
mid.save('generated.midi')

In [64]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)
delta = 300
for _ in test:
    n = np.random.randint(12) + 60
    track.append(mido.Message('note_on', note=n, velocity=64, time=0))
    track.append(mido.Message('note_on', note=n, velocity=0, time=delta))
mid.save('random.midi')